# I. Graph convolutional networks

In this notebook, we'll show how to build graph convolutional networks (GCNs) 
using the differential structures exposed by `topos`. 

In [1]:
import torch
import topos

## I.1. Graphs and complexes

In the following, we denote by $\Omega = \{1, \dots, N_{\rm vtx}\}$ a finite set 
of vertices. 

A 1-graph $G$ is usually defined by a set of edges contained in 
$\Omega \times \Omega$ with set of vertices is $\Omega$ 
i.e. the boundary of every edge is a vertex of the graph.
This is a particular case of simplicial complex (dimension 1), and the 
reason why we shall use the `Complex` class. 

__Definition 1.__
A _simplicial complex_ $K \subset {\mathcal P}(\Omega)$ is a collection of regions 
such that for all $a \in K$, every $b \subset a$ is also in $K$.
The _dimension_ of $K$ is one plus the maximal size of regions $a \in K$, also called _faces_ of $K$. 


In [2]:
""" Creation of 1D complexes """

from topos import Complex

#-- Complex constructor: ensure to provide all vertices! --
G = Complex([[0, 1, 2], [[0, 1], [0, 2], [1, 2]]])

#-- Simplicial closure : subfaces i.e. vertices are computed --
G = Complex.simplicial([[0, 1], [0, 2], [1, 2]])

The sets of vertices and edges can be accessed by `G[0]` and `G[1]` respectively. They are implemented as `Sheaf` instances, as we shall discuss in the next section. 

Let us for now briefly review the different `Field` types associated to $G, G_0$ and $G_1$. Fields are scalar valued functions on their domains, i.e. vectors of dimension `field.domain.size`. The type constructor is obtained by wrapping `torch.Tensor` types inside a custom Wrap monad
(see [fp.Tens](https://github.com/opeltre/fp/fp/instances/tens.py)).
Field types will further allow us to define a type for linear maps from $G$-fields to $G'$-fields, etc.

In [3]:
""" Field types """

from topos import Field

assert isinstance(Field(G), type)

#----- G.Field(degree=None) ------
assert G.Field()  == Field(G)
assert G.Field(0) == Field(G[0])
assert G.Field(1) == G[1].Field()

assert isinstance(G.randn(0), G.Field(0))
Field

Functor : Field

A `Field(G)` instance of particular interest to understand the data structure is `G.range()`: its wrapped vector attribute coincides with `torch.arange(G.size)` and therefore maps every point of the domain to its index in `G`.

In [4]:
""" Field creation """

#-- Index map on G --
x = G.range()

#-- Index map on G[0]
x0 = G.range(0)

#-- Index map on G[1]
x1 = G[1].range()

#-- x = x0 | x1 + G.begin[1] | ... --
assert (x0.data == x.data[:G.begin[1]]).prod()
assert (x1.data == x.data[G.begin[1]:] - G.begin[1]).prod()
x

Field G :  0 :  [0] :        0
                [1] :        1
                [2] :        2
               
           1 :  [0, 1] :        3
                [0, 2] :        4
                [1, 2] :        5
               
          

For now fields are only scalar-valued functions on the sets of edges and vertices. It is more interesting to allow for arbitrary numbers of vertex and edge features. This is what we'll introduce in section I.3. 

## I.3. Functor values

There are many possible ways one may assign degrees of freedom to regions. To build vanilla GCNs, we may assume that edges and vertices all have the same degrees of freedom, i.e. fibers $F_i$, $F_j$, $F_{ij}$, ... are all isomorphic 
to a common finite set $F$.

To compute the differential $d$ and codifferential $\delta$ in a functor-valued graph, it is required that:
- either edge features $F_{ij}$ can be mapped to vertex features $F_i, F_j$
- either vertex features $F_i, F_j$ can be mapped to edge features $F_{ij}$ 

The `ConstantFunctor` subclass will yield a single object $F$ and identity arrows. 
It is in practice equivalent to:


In [5]:
""" Constant feature spaces """

def ConstantFunctor(shape):
    """ Single shape interpreted as a finite set + identities """        
    #----- topos.Functor(obj_map, hom_map)
    return topos.Functor(lambda _: shape, lambda _: lambda x: x)

The `Complex` constructor accepts an optional `functor` second argument.
Note how index ranges have been expanded. 

In [6]:
""" Functor valued complex """

F  = ConstantFunctor([3])
GF = Complex(G, F)

GF.range()

Field G :  0 :  [0] :        [0, 1, 2]
                [1] :        [3, 4, 5]
                [2] :        [6, 7, 8]
               
           1 :  [0, 1] :        [ 9, 10, 11]
                [0, 2] :        [12, 13, 14]
                [1, 2] :        [15, 16, 17]
               
          

With a constant functor, the differential and codifferential will simply add and subtract field components feature-wise, e.g.

In [7]:
""" Functor-valued differential """

GF.diff(0) @ GF.range(0)

Field Ω :  [0, 1] :        [3., 3., 3.]
           [0, 2] :        [6., 6., 6.]
           [1, 2] :        [3., 3., 3.]
          

In [8]:
""" Functor-valued codifferential """

GF.codiff(1) @ GF.ones(1)

Field Ω :  [0] :        [-2., -2., -2.]
           [1] :        [0., 0., 0.]
           [2] :        [2., 2., 2.]
          

In [9]:
help(F.fmap)

Help on method fmap in module topos.base.functor:

fmap(f) method of topos.base.functor.Functor instance
    Map a pair of indices to coordinates.

